# Dataset Sources report

This notebook produces a table showing the source and count of entities in a given dataset.

It needs 

1. A recently downloaded copy of the entity datasette from https://datasette.planning.data.gov.uk/entity which it uses as input. You can see the input and output file names in the next section of this notebook.
1. the name of the dataset you are interested in, again below.

It writes the result to a csv file with the given name.

In [113]:
dataset_name="conservation-area"

source_file = "/mnt/c/Users/MarkSmith/Downloads/entity_2023_09_12.sqlite3" # or whatever you called your download.
dest_file=  F"/mnt/c/Users/MarkSmith/Downloads/entity_2023_09_12_{dataset_name}.csv" # or wherever you want your output



First time to run is a bit slow as it indexes the entity table.

In [114]:
import sqlite3
import pandas as pd

# Create the connection
cnx = sqlite3.connect(source_file)

cursor = cnx.cursor()

cnx.execute("CREATE INDEX if not exists idx_entity ON entity(dataset)") 
cnx.execute("CREATE INDEX if not exists idx_organisation_entity ON entity(organisation_entity)") 


In [115]:
lpas = pd.read_sql_query("SELECT entity as id, naME, REFERENCE FROM Entity where dataset = 'local-authority'", cnx)
lpas['id'] = lpas['id'].astype('Int64', errors='ignore') # there's a type error when you concet later if you don't do this.

In [116]:
query = F"""
SELECT organisation_entity as id, count (*) 
from Entity 
where dataset = '{dataset_name}' 
and organisation_entity != ""
group by 1
order by id 
"""

rows = pd.read_sql_query(  query, cnx)
rows['id'] = rows['id'].astype('Int64', errors='raise')

In [117]:
result = pd.merge(lpas, rows, on= 'id',  how = "inner")

result

,id,name,reference,count (*)
0,33,Babergh District Council,BAB,12
1,42,London Borough of Brent,BEN,23
2,43,London Borough of Bexley,BEX,12
3,48,London Borough of Barnet,BNE,11
4,51,Bolton Metropolitan Borough Council,BOL,25
...,...,...,...,...
60,366,London Borough of Waltham Forest,WFT,1
61,376,London Borough of Wandsworth,WND,18
62,383,West Oxfordshire District Council,WOX,2
63,387,City of Westminster,WSM,2


In [118]:
result.to_csv(dest_file)

print (F"\nReport saved to {dest_file}")


Report saved to /mnt/c/Users/MarkSmith/Downloads/entity_2023_09_12_conservation-area.csv
